# 层和块

我们先回顾一下多层感知机

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.2275, -0.0931,  0.0570,  0.0855, -0.0524,  0.0325,  0.1676, -0.1014,
          0.0794, -0.1586],
        [ 0.2249, -0.0549, -0.0864, -0.1089, -0.1159, -0.0089,  0.1040, -0.2234,
          0.1652, -0.2738]], grad_fn=<AddmmBackward0>)

`nn.Sequential`定义了一种特殊的`Module`

自定义块

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

实例化多层感知机的层，然后在每次调用前向传播函数时调用这些层

In [3]:
net = MLP()
net(X)

tensor([[ 0.1981,  0.1234,  0.0592,  0.0819,  0.0648,  0.3781,  0.1214,  0.1906,
         -0.0062,  0.1415],
        [ 0.2254,  0.0934, -0.0539,  0.1771,  0.0199,  0.2174,  0.2251,  0.3334,
         -0.0966,  0.2420]], grad_fn=<AddmmBackward0>)

顺序块

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0270, -0.0326, -0.0883,  0.0289,  0.0240, -0.2199,  0.1712, -0.1201,
          0.1149,  0.1823],
        [-0.0254, -0.0283, -0.1461,  0.1541,  0.2295, -0.3174, -0.0860,  0.0275,
          0.0732,  0.1548]], grad_fn=<AddmmBackward0>)

在前向传播函数中执行代码

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(-0.2160, grad_fn=<SumBackward0>)

混合搭配各种组合块的方法

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.4822, grad_fn=<SumBackward0>)